# HMMS POS Tagging

In [2]:
from __future__ import print_function
import nltk
import sys
from nltk.corpus import brown
from nltk.corpus import nps_chat
from nltk.corpus import conll2000

In [6]:
nps_chat.tagged_posts()

[[(u'now', 'RB'), (u'im', 'PRP'), (u'left', 'VBD'), (u'with', 'IN'), (u'this', 'DT'), (u'gay', 'JJ'), (u'name', 'NN')], [(u':P', 'UH')], ...]

In [7]:
conll2000.tagged_sents()

[[(u'Confidence', u'NN'), (u'in', u'IN'), (u'the', u'DT'), (u'pound', u'NN'), (u'is', u'VBZ'), (u'widely', u'RB'), (u'expected', u'VBN'), (u'to', u'TO'), (u'take', u'VB'), (u'another', u'DT'), (u'sharp', u'JJ'), (u'dive', u'NN'), (u'if', u'IN'), (u'trade', u'NN'), (u'figures', u'NNS'), (u'for', u'IN'), (u'September', u'NNP'), (u',', u','), (u'due', u'JJ'), (u'for', u'IN'), (u'release', u'NN'), (u'tomorrow', u'NN'), (u',', u','), (u'fail', u'VB'), (u'to', u'TO'), (u'show', u'VB'), (u'a', u'DT'), (u'substantial', u'JJ'), (u'improvement', u'NN'), (u'from', u'IN'), (u'July', u'NNP'), (u'and', u'CC'), (u'August', u'NNP'), (u"'s", u'POS'), (u'near-record', u'JJ'), (u'deficits', u'NNS'), (u'.', u'.')], [(u'Chancellor', u'NNP'), (u'of', u'IN'), (u'the', u'DT'), (u'Exchequer', u'NNP'), (u'Nigel', u'NNP'), (u'Lawson', u'NNP'), (u"'s", u'POS'), (u'restated', u'VBN'), (u'commitment', u'NN'), (u'to', u'TO'), (u'a', u'DT'), (u'firm', u'NN'), (u'monetary', u'JJ'), (u'policy', u'NN'), (u'has', u'VBZ')

In [8]:
brown.tagged_sents()

[[(u'The', u'AT'), (u'Fulton', u'NP-TL'), (u'County', u'NN-TL'), (u'Grand', u'JJ-TL'), (u'Jury', u'NN-TL'), (u'said', u'VBD'), (u'Friday', u'NR'), (u'an', u'AT'), (u'investigation', u'NN'), (u'of', u'IN'), (u"Atlanta's", u'NP$'), (u'recent', u'JJ'), (u'primary', u'NN'), (u'election', u'NN'), (u'produced', u'VBD'), (u'``', u'``'), (u'no', u'AT'), (u'evidence', u'NN'), (u"''", u"''"), (u'that', u'CS'), (u'any', u'DTI'), (u'irregularities', u'NNS'), (u'took', u'VBD'), (u'place', u'NN'), (u'.', u'.')], [(u'The', u'AT'), (u'jury', u'NN'), (u'further', u'RBR'), (u'said', u'VBD'), (u'in', u'IN'), (u'term-end', u'NN'), (u'presentments', u'NNS'), (u'that', u'CS'), (u'the', u'AT'), (u'City', u'NN-TL'), (u'Executive', u'JJ-TL'), (u'Committee', u'NN-TL'), (u',', u','), (u'which', u'WDT'), (u'had', u'HVD'), (u'over-all', u'JJ'), (u'charge', u'NN'), (u'of', u'IN'), (u'the', u'AT'), (u'election', u'NN'), (u',', u','), (u'``', u'``'), (u'deserves', u'VBZ'), (u'the', u'AT'), (u'praise', u'NN'), (u'and'

The structure is the same, so we take the POS tags(using only the first two char of the POS tag)

In [9]:
brown_word_tags = []
for sent in brown.tagged_sents():
    brown_word_tags.append(("START", "START"))
    brown_word_tags.extend([(tag[:2], word) for (word, tag) in sent])
    brown_word_tags.append(("END", "END"))
    
conll_word_tags = []    
for sent in conll2000.tagged_sents():
    conll_word_tags.append(("START", "START"))
    conll_word_tags.extend([(tag[:2], word) for (word, tag) in sent])
    conll_word_tags.append(("END", "END"))
    
nps_word_tags = []
for sent in nps_chat.tagged_posts():
    nps_word_tags.append(("START", "START"))
    nps_word_tags.extend([(tag[:2], word) for (word, tag) in sent])
    nps_word_tags.append(("END", "END"))

Corpus Stats

In [12]:
print("Number of instances in Brown corpus: ", len(brown.tagged_sents()))
print("Number of instances in CONL2000 corpus: ", len(nps_chat.tagged_posts()))
print("Number of instances in NPS_chat corpus: ", len(conll2000.tagged_sents()))

Number of instances in Brown corpus:  57340
Number of instances in CONL2000 corpus:  10567
Number of instances in NPS_chat corpus:  10948


Training using Brown

In [13]:
# conditional frequency distribution
cfd_tagwords = nltk.ConditionalFreqDist(brown_word_tags)
# conditional probability distribution usando MLE
cpd_tagwords = nltk.ConditionalProbDist(cfd_tagwords, nltk.MLEProbDist)

In [14]:
brown_tags = [tag for (tag, word) in brown_word_tags]

In [15]:
cfd_tags= nltk.ConditionalFreqDist(nltk.bigrams(brown_tags))
cpd_tags = nltk.ConditionalProbDist(cfd_tags, nltk.MLEProbDist)

In [55]:
conll2000.sents()[3]

[u'This',
 u'has',
 u'increased',
 u'the',
 u'risk',
 u'of',
 u'the',
 u'government',
 u'being',
 u'forced',
 u'to',
 u'increase',
 u'base',
 u'rates',
 u'to',
 u'16',
 u'%',
 u'from',
 u'their',
 u'current',
 u'15',
 u'%',
 u'level',
 u'to',
 u'defend',
 u'the',
 u'pound',
 u',',
 u'economists',
 u'and',
 u'foreign',
 u'exchange',
 u'market',
 u'analysts',
 u'say',
 u'.']

In [56]:
distinct_tags = set(brown_tags)

#sentence = ["I", "want", "to", "race" ]
#sentence = ["I", "saw", "her", "duck" ]
sentence = conll2000.sents()[3]
sentlen = len(sentence)

viterbi = []
backpointer = [ ]

# Inicializacion de las variables de viterbi_1 v_s(1)
first_viterbi = { }
first_backpointer = { }
for tag in distinct_tags:
    # don't record anything for the START tag
    if tag == "START": continue
    first_viterbi[ tag ] = cpd_tags["START"].prob(tag) * cpd_tagwords[tag].prob( sentence[0] )
    first_backpointer[ tag ] = "START"

print(first_viterbi)
print(first_backpointer)

{u'DO': 0.0, u'BE': 0.0, u'WD': 0.0, u'WQ': 0.0, u'FW': 0.0, u'IN': 0.0, u'HV': 0.0, u'VB': 0.0, u"''": 0.0, u')-': 0.0, u'WR': 0.0, u'JJ': 0.0, u'WP': 0.0, u'DT': 0.0033218181276236437, u',-': 0.0, u'NI': 0.0, u'PP': 0.0, u'RP': 0.0, u'.-': 0.0, u'NN': 0.0, u')': 0.0, u'(': 0.0, u'*': 0.0, u',': 0.0, u'.': 0.0, u'TO': 0.0, u'RB': 0.0, u'NP': 0.0, u'RN': 0.0, u':': 0.0, u'PN': 0.0, u'``': 0.0, u'AB': 0.0, 'END': 0.0, u'CC': 0.0, u'CD': 0.0, u'*-': 0.0, u'AT': 0.0, u'(-': 0.0, u'CS': 0.0, u'NR': 0.0, u'MD': 0.0, u':-': 0.0, u"'": 0.0, u'AP': 0.0, u'--': 0.0, u'OD': 0.0, u'UH': 0.0, u'QL': 0.0, u'EX': 0.0}
{u'DO': 'START', u'BE': 'START', u'WD': 'START', u'WQ': 'START', u'FW': 'START', u'IN': 'START', u'HV': 'START', u'VB': 'START', u"''": 'START', u')-': 'START', u'WR': 'START', u'JJ': 'START', u'WP': 'START', u'DT': 'START', u',-': 'START', u'NI': 'START', u'PP': 'START', u'RP': 'START', u'.-': 'START', u'NN': 'START', u')': 'START', u'(': 'START', u'*': 'START', u',': 'START', u'.': '

In [57]:
viterbi.append(first_viterbi)
backpointer.append(first_backpointer)

currbest = max(first_viterbi.keys(), key = lambda tag: first_viterbi[ tag ])
print( "Word", "'" + sentence[0] + "'", "current best two-tag sequence:", 
      first_backpointer[ currbest], currbest)

Word 'This' current best two-tag sequence: START DT


In [58]:
for wordindex in range(1, len(sentence)):
    this_viterbi = { }
    this_backpointer = { }
    prev_viterbi = viterbi[-1]
    
    for tag in distinct_tags:
        if tag == "START": continue
        best_previous = max(prev_viterbi.keys(),
                            key = lambda prevtag: \
            prev_viterbi[ prevtag ] * cpd_tags[prevtag].prob(tag) * 
                            cpd_tagwords[tag].prob(sentence[wordindex]))

        this_viterbi[ tag ] = prev_viterbi[ best_previous] * \
            cpd_tags[ best_previous ].prob(tag) * cpd_tagwords[ tag].prob(sentence[wordindex])
        this_backpointer[ tag ] = best_previous

    currbest = max(this_viterbi.keys(), key = lambda tag: this_viterbi[ tag ])
    print( "Word", "'" + sentence[ wordindex] + "'", "current best two-tag sequence:", 
          this_backpointer[ currbest], currbest)
    viterbi.append(this_viterbi)
    backpointer.append(this_backpointer)

prev_viterbi = viterbi[-1]
best_previous = max(prev_viterbi.keys(),
                    key = lambda prevtag: prev_viterbi[ prevtag ] * 
                    cpd_tags[prevtag].prob("END"))

prob_tagsequence = prev_viterbi[ best_previous ] * cpd_tags[ best_previous].prob("END")

best_tagsequence = [ "END", best_previous ]
backpointer.reverse()

current_best_tag = best_previous
for bp in backpointer:
    best_tagsequence.append(bp[current_best_tag])
    current_best_tag = bp[current_best_tag]

best_tagsequence.reverse()
print( "The sentence was:", end = " ")
for w in sentence: print( w, end = " ")
print("\n")
print( "The best tag sequence is:", end = " ")
for t in best_tagsequence: print (t, end = " ")
print("\n")
print( "The probability of the best tag sequence is:", prob_tagsequence)

Word 'has' current best two-tag sequence: DT HV
Word 'increased' current best two-tag sequence: HV VB
Word 'the' current best two-tag sequence: VB AT
Word 'risk' current best two-tag sequence: AT NN
Word 'of' current best two-tag sequence: NN IN
Word 'the' current best two-tag sequence: IN AT
Word 'government' current best two-tag sequence: AT NN
Word 'being' current best two-tag sequence: NN BE
Word 'forced' current best two-tag sequence: BE VB
Word 'to' current best two-tag sequence: VB TO
Word 'increase' current best two-tag sequence: TO VB
Word 'base' current best two-tag sequence: VB NN
Word 'rates' current best two-tag sequence: NN NN
Word 'to' current best two-tag sequence: NN IN
Word '16' current best two-tag sequence: IN CD
Word '%' current best two-tag sequence: DO DO
Word 'from' current best two-tag sequence: DO DO
Word 'their' current best two-tag sequence: DO DO
Word 'current' current best two-tag sequence: DO DO
Word '15' current best two-tag sequence: DO DO
Word '%' curr

Put it all in a function:

In [20]:
def train_hmms(tags, cpd_tags, cpd_tagwords, sentence):
    #distinct_tags = set(tags)

    #sentence = ["I", "want", "to", "race" ]
    #sentence = ["I", "saw", "her", "duck" ]
    sentlen = len(sentence)

    viterbi = []
    backpointer = [ ]

    # Inicializacion de las variables de viterbi_1 v_s(1)
    first_viterbi = { }
    first_backpointer = { }
    for tag in distinct_tags:
        # don't record anything for the START tag
        if tag == "START": continue
        first_viterbi[ tag ] = cpd_tags["START"].prob(tag) * cpd_tagwords[tag].prob( sentence[0] )
        first_backpointer[ tag ] = "START"

    viterbi.append(first_viterbi)
    backpointer.append(first_backpointer)

    currbest = max(first_viterbi.keys(), key = lambda tag: first_viterbi[tag])

    for wordindex in range(1, len(sentence)):
        this_viterbi = {}
        this_backpointer = {}
        prev_viterbi = viterbi[-1]

        for tag in distinct_tags:
            if tag == "START": continue
            best_previous = max(prev_viterbi.keys(),
                                key = lambda prevtag: \
                prev_viterbi[ prevtag ] * cpd_tags[prevtag].prob(tag) * 
                                cpd_tagwords[tag].prob(sentence[wordindex]))

            this_viterbi[ tag ] = prev_viterbi[ best_previous] * \
                cpd_tags[ best_previous ].prob(tag) * cpd_tagwords[ tag].prob(sentence[wordindex])
            this_backpointer[ tag ] = best_previous

        currbest = max(this_viterbi.keys(), key = lambda tag: this_viterbi[ tag ])
        viterbi.append(this_viterbi)
        backpointer.append(this_backpointer)

    prev_viterbi = viterbi[-1]
    best_previous = max(prev_viterbi.keys(),
                        key = lambda prevtag: prev_viterbi[ prevtag ] * 
                        cpd_tags[prevtag].prob("END"))

    prob_tagsequence = prev_viterbi[ best_previous ] * cpd_tags[ best_previous].prob("END")

    best_tagsequence = [ "END", best_previous ]
    backpointer.reverse()

    current_best_tag = best_previous
    for bp in backpointer:
        best_tagsequence.append(bp[current_best_tag])
        current_best_tag = bp[current_best_tag]

    best_tagsequence.reverse()
    return prob_tagsequence, best_tagsequence

In [29]:
distinct_tags = set(brown_tags)

In [41]:
nps_chat.posts()[20]

[u'i', u"'ll", u'thunder', u'clap', u'your', u'ass', u'.']

In [39]:
brown.sents()[15]

[u'The',
 u'jurors',
 u'said',
 u'they',
 u'realize',
 u'``',
 u'a',
 u'proportionate',
 u'distribution',
 u'of',
 u'these',
 u'funds',
 u'might',
 u'disable',
 u'this',
 u'program',
 u'in',
 u'our',
 u'less',
 u'populous',
 u'counties',
 u"''",
 u'.']

In [44]:
train_hmms(distinct_tags, cpd_tags, cpd_tagwords, conll2000.sents()[0])

(0.0,
 ['START',
  u'NN',
  u'IN',
  u'AT',
  u'NN',
  u'BE',
  u'BE',
  u'BE',
  u'BE',
  u'BE',
  u'BE',
  u'BE',
  u'BE',
  u'BE',
  u'BE',
  u'BE',
  u'BE',
  u'BE',
  u'BE',
  u'BE',
  u'BE',
  u'BE',
  u'BE',
  u'BE',
  u'BE',
  u'BE',
  u'BE',
  u'BE',
  u'BE',
  u'BE',
  u'BE',
  u'BE',
  u'BE',
  u'BE',
  u'BE',
  u'BE',
  u'BE',
  u'BE',
  'END'])

In [38]:
train_hmms(distinct_tags, cpd_tags, cpd_tagwords, brown.sents()[15])

(3.829128179255946e-69,
 ['START',
  u'AT',
  u'NN',
  u'VB',
  u'PP',
  u'VB',
  u'``',
  u'AT',
  u'JJ',
  u'NN',
  u'IN',
  u'DT',
  u'NN',
  u'MD',
  u'VB',
  u'DT',
  u'NN',
  u'IN',
  u'PP',
  u'QL',
  u'JJ',
  u'NN',
  u"''",
  u'.',
  'END'])

In [28]:
nps_chat.ta()

[[(u'now', 'RB'), (u'im', 'PRP'), (u'left', 'VBD'), (u'with', 'IN'), (u'this', 'DT'), (u'gay', 'JJ'), (u'name', 'NN')], [(u':P', 'UH')], ...]

# 1. Training and Testing splits

# 2. Corpora model training

# 3. Evaluation